In [1]:
import gc
import gensim
import nltk
from nltk.corpus import stopwords
import numpy as np
import os
import pandas as pd
import pyLDAvis.gensim
import random
import re
import string
import time
from tqdm import tqdm

from hughes_utilities import *

# turning off logging for readability purposes

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
full_table = pd.read_csv('/home/ubuntu/full_text_table.csv')
print('Full table has {} rows, {} columns'.format(full_table.shape[0], full_table.shape[1]))

Full table has 1316579 rows, 22 columns


**Pretty big dataset - going to reduce this in size for usability purposes**

In [3]:
halved_df = cut_table(full_table, keep=100000)

In [4]:
halved_df.shape

(100000, 22)

In [5]:
# free some memory
del full_table

In [6]:
cols = ['authors', 'summary', 'tags', 'title', 'title_detail']

df = trim_table(dataframe=halved_df, columns_to_keep=cols)

In [7]:
print('Summary: \n\n\t{}'.format(df.summary.iloc[4321]))
print('\n\n')
print('Tags: \n\n\t{}'.format(df.tags.iloc[4321]))

Summary: 

	We present the physical properties of the eclipsing binary V2281 Cyg, which
shows a light-time effect due to a supposed tertiary component from its eclipse
timing variation according to the $Kepler$ observations. The high-resolution
spectra and $BVR$ photometric data of the system were obtained at Bohyunsan
Optical Astronomy Observatory and Mount Lemmon Optical Astronomy Observatory,
respectively. To determine the fundamental parameters of the eclipsing pair and
its circumbinary object, we simultaneously analyzed the radial velocities,
light curves, and eclipse times including the $Kepler$ data. The masses and
radii for the primary and secondary stars were determined with accuracy levels
of approximately 2\% and 1\%, respectively, as follows: $M_{1} = 1.61 \pm 0.04$
M$_\odot$ and $M_{2} = 1.60 \pm 0.04$ M$_\odot$, $R_{1} = 1.94 \pm 0.02$
R$_\odot$ and $R_{2} = 1.93 \pm 0.02$ R$_\odot$. If its orbit is coplanar with
the eclipsing binary, the period and semimajor axis of the 

In [8]:
#df.summary = df.summary.apply(lambda x: clean_text(x, tokenized=True))
tqdm.pandas()
df.summary = df.summary.progress_apply(lambda x: clean_text(x, tokenized=True))

100%|██████████| 100000/100000 [00:31<00:00, 3129.32it/s]


In [9]:
#tqdm.pandas()
#df.summary = df.summary.progress_apply(lambda x: remove_stopwords(x))

df.summary = df.summary.apply(lambda x: remove_stopwords(x))

In [10]:
texts = list(df.summary)

**Create dictionary and corpus:**

In [11]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

timestr = time.strftime("%Y%m%d")
gensim.corpora.MmCorpus.serialize('./arxiv_text_{}.mm'.format(timestr), corpus)
corpus = gensim.corpora.MmCorpus('./arxiv_text_{}.mm'.format(timestr))

In [12]:
num_topics = 20
chunksize = 2500
passes = 5
iterations = 100
eval_every = 1

_ = dictionary[0] # this is to initialize dictionary
id2word = dictionary.id2token

In [13]:
%time 
model = gensim.models.LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, alpha='auto', eta='auto', \
                                     iterations=iterations, num_topics=num_topics,\
                                     passes=passes, eval_every=eval_every)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.77 µs


/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: invalid value encountered in log
  diff = np.log(self.expElogbeta)


In [14]:
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(model, corpus, dictionary, R=15)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.021148 -0.158262       1        1  11.883286
7      0.014086 -0.122362       2        1  10.038625
1      0.238175 -0.100437       3        1   7.590487
8      0.178474 -0.156048       4        1   7.303280
3      0.014055 -0.126823       5        1   7.035790
11    -0.147537 -0.032443       6        1   6.624125
9     -0.133232 -0.067426       7        1   6.250048
10    -0.085544  0.027154       8        1   5.910592
6     -0.140387 -0.019323       9        1   5.502818
18    -0.097991 -0.044843      10        1   5.233946
4     -0.176590 -0.058671      11        1   4.773161
15    -0.127739 -0.011230      12        1   4.332935
5      0.266405 -0.083302      13        1   3.749552
19    -0.087856 -0.005098      14        1   3.388243
12     0.186734  0.014934      15        1   3.148400
16    -0.077209  0.074615      16        1   2.420855
14     0.048601  0.176041      17        1   1.652526
0      0.016632  0.163713      18        1   1.185789
13     0.056885  0.263098      19        1   1.025626
17     0.075187  0.266715      20        1   0.949917, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
187    Default  37142.000000          model  37142.000000  15.0000  15.0000
1821   Default  11263.000000        network  11263.000000  14.0000  14.0000
3589   Default  10989.000000       networks  10989.000000  13.0000  13.0000
1151   Default  26450.000000           data  26450.000000  12.0000  12.0000
2816   Default  12522.000000              n  12522.000000  11.0000  11.0000
1709   Default  13280.000000      algorithm  13280.000000  10.0000  10.0000
535    Default  19352.000000        problem  19352.000000   9.0000   9.0000
435    Default  15964.000000        quantum  15964.000000   8.0000   8.0000
5515   Default   8248.000000          graph   8248.000000   7.0000   7.0000
132    Default  16598.000000          field  16598.000000   6.0000   6.0000
455    Default  15583.000000         models  15583.000000   5.0000   5.0000
65     Default  14229.000000          phase  14229.000000   4.0000   4.0000
211    Default   9005.000000       magnetic   9005.000000   3.0000   3.0000
4464   Default   8387.000000         matrix   8387.000000   2.0000   2.0000
335    Default  22360.000000         system  22360.000000   1.0000   1.0000
3426    Topic1   1862.853027      tunneling   1863.278198   2.1298  -6.1866
2417    Topic1   1468.597168          dirac   1468.996094   2.1298  -6.4244
9599    Topic1   1957.636963           hall   1958.228882   2.1297  -6.1370
17284   Topic1   1522.800659          ising   1523.294434   2.1297  -6.3882
5118    Topic1   1319.018433      meanfield   1319.473267   2.1297  -6.5318
25050   Topic1   1498.013916            dot   1498.655029   2.1296  -6.4046
41809   Topic1    765.459106          qubit    765.883911   2.1295  -7.0760
4151    Topic1   1011.508240        barrier   1012.078003   2.1295  -6.7973
14310   Topic1    618.011963          loops    618.376770   2.1294  -7.2900
8336    Topic1    776.639221       manybody    777.097961   2.1294  -7.0615
13427   Topic1    738.317932        soliton    738.766663   2.1294  -7.1121
14452   Topic1    594.895874       kinetics    595.296204   2.1294  -7.3281
1610    Topic1    609.685852     superfluid    610.119263   2.1293  -7.3035
42469   Topic1    575.302429      plasmonic    575.716797   2.1293  -7.3616
4195    Topic1    509.396606      stiffness    509.773560   2.1293  -7.4832
...        ...           ...            ...           ...      ...      ...
24941  Topic19    322.506561             rf    322.964630   4.5784  -5.4905
5354   Topic19    526.074890        nearest    585.875977   4.4722  -5.0012
5355   Topic19    531.235840      neighbors    612.118225   4.4381  -4.9914
5321   Topic19    634.337646     